In [24]:
from MPC_problem import B1, B2, B3, B5, Amld, hybrid_fhocp_milp, simulate_system 
from transformer_mpc import try_model, run_optimization, normalize_tensor, global_mean, global_std, load_model_and_config, compute_global_stats
import numpy as np
import torch
import gurobipy as gp


cbuy, csell, cprod, power_load, power_res = np.load('price_load_res_profiles.npy', allow_pickle=True)
net_power =  power_load - power_res




In [25]:
def closed_loop_mpc(x0, N, cbuy, csell, cprod, net_power, time_steps, B1, B2, B3, B5, Amld, index):
    n=Amld.shape[1]; m=B1.shape[1]; N_bin = B2.shape[1]; N_z = B3.shape[1]
    objective_values = []  # List to store the objective values

    x_current = x0
    for i in range(time_steps):
        # Slice the data for the current iteration
        cbuy_tmp = cbuy[index:index+N+1]
        csell_tmp = csell[index:index+N+1]
        cprod_tmp = cprod[index:index+N+1]
        power_res_tmp = power_res[index:index+N+1]
        power_load_tmp = power_load[index:index+N+1]
        net_power_tmp =  power_load_tmp - power_res_tmp
        
        # Solve the MPC problem
        mdl = hybrid_fhocp_milp(x0, N, net_power_tmp, cbuy_tmp, csell_tmp, cprod_tmp)
        objective_values.append(mdl.ObjVal)  # Save the objective value if the model is optimal

        # Extract control actions for the first time step correctly
        var_index = 0
        x_vars = mdl.getVars()[var_index:var_index + (N+1)*n]
        var_index += (N+1)*n
        z_vars = mdl.getVars()[var_index:var_index + N*N_z]
        var_index += N*N_z
        u_vars = mdl.getVars()[var_index:var_index + N*m]
        var_index += N*m
        delta_vars = mdl.getVars()[var_index:var_index + N*N_bin]
        
        #x_next = np.array([x_vars[j].X for j in range(n)])  # Updated state

        u0 = np.array([u_vars[j].X for j in range(m)])  # First control action
        delta0 = np.array([delta_vars[j].X for j in range(N_bin)])  # First delta
        z0 = np.array([z_vars[j].X for j in range(N_z)])  # First z
        
        x_next = simulate_system(x_current, u0, delta0, z0, B1, B2, B3, B5, Amld)

        # Update current state and index for the next iteration
        x_current = x_next
        index += 1  # Move to the next data point, ensures new data is used in each iteration
        #print(f"Time Step {i+1}, State: {x_current}")
    avg_cost = np.mean(objective_values) if objective_values else None
    print(f"Average Objective Value: {avg_cost}")
    return avg_cost

In [26]:
# model, config_model = load_model_and_config("transformer_model.pth")




In [27]:
def simulate_system(x, u, delta, z, B1, B2, B3, B5, Amld):
    return Amld @ x + B1 @ u + B2 @ delta + B3 @ z + B5

def normalize_tensor(tensor, mean, std):
    return (tensor - mean) / std


def closed_loop_mpc_transformer1(x0, N, cbuy, csell, cprod, net_power, time_steps, B1, B2, B3, B5, Amld, models, global_mean, global_std, index):
    n=Amld.shape[1]; m=B1.shape[1]; N_bin = B2.shape[1]; N_z = B3.shape[1]
    objective_values = []  # List to store objective values

    fallback_count = 0
    x_current = x0  # Initialize the current state with the initial state

    for i in range(time_steps):
        # Slice the data for the current iteration
        cbuy_tmp = cbuy[index:index+N].astype(np.float32)
        csell_tmp = csell[index:index+N].astype(np.float32)
        cprod_tmp = cprod[index:index+N].astype(np.float32)
        power_res_tmp = power_res[index:index+N].astype(np.float32)
        power_load_tmp = power_load[index:index+N].astype(np.float32)
        net_power_tmp = power_load_tmp - power_res_tmp

        # Prepare the input tensor for the transformer model
        x_current_tensor = torch.tensor(x_current, dtype=torch.float32)
        if x_current_tensor.dim() == 1:
            x_current_tensor = x_current_tensor.unsqueeze(0)

        x_current_repeated = x_current_tensor.repeat(N, 1)  # Repeat x0 for each row in the sequence

        src_tensor = torch.cat([
            torch.tensor(cbuy_tmp).unsqueeze(-1),  # Adding an extra dimension for feature alignment
            torch.tensor(csell_tmp).unsqueeze(-1),
            torch.tensor(cprod_tmp).unsqueeze(-1),
            torch.tensor(net_power_tmp).unsqueeze(-1),
            x_current_repeated # Adding an extra dimension for feature alignment
        ], dim=-1)  # Concatenate along the last dimension to combine features

        #global_mean, global_std = compute_global_stats(val_dataset)
        
        src_tensor = src_tensor.unsqueeze(0)  # Add batch dimension
        src_tensor_normalized = normalize_tensor(src_tensor, global_mean, global_std)

        # Model prediction with fallback strategy
        optimization_found = False
        for model in models:
            model.eval()
            delta_predicted = try_model(model, src_tensor_normalized)

            # Run optimization with the first delta_predicted to get u and z
            mdl_pred = run_optimization(delta_predicted, src_tensor, model.__class__.__name__)
            if mdl_pred and mdl_pred.status == gp.GRB.OPTIMAL:
                optimization_found = True
                objective_values.append(mdl_pred.ObjVal)  # Save objective value

                var_index = 0
                x_vars = mdl_pred.getVars()[var_index:var_index + (N+1)*n]
                var_index += (N+1)*n
                z_vars = mdl_pred.getVars()[var_index:var_index + N*N_z]
                var_index += N*N_z
                u_vars = mdl_pred.getVars()[var_index:var_index + N*m]
                var_index += N*m

                u0 = np.array([u_vars[j].X for j in range(m)])  # First control action
                z0 = np.array([z_vars[j].X for j in range(N_z)])  # First z

                # Simulate system response
                x_next = simulate_system(x_current, u0, delta_predicted[0, :], z0, B1, B2, B3, B5, Amld)

                
                # Update current state and index for the next iteration
                x_current = x_next  # Update current state for the next iteration
                index += 1  # Move to the next data point to ensure data shifts

               # print(f"Time step {i+1}, State: {x_current}")
                break
                
        if not optimization_found:
            #print(f"Iteration {i+1}, No feasible solution found with any model, falling back to hybrid MPC")
            mdl_fallback = hybrid_fhocp_milp(x0, N, net_power_tmp, cbuy_tmp, csell_tmp, cprod_tmp)

            fallback_count += 1
            if mdl_fallback and mdl_fallback.status == gp.GRB.OPTIMAL:
                objective_values.append(mdl_fallback.ObjVal)  # Save objective value from hybrid fallback

                var_index = 0
                x_vars = mdl_fallback.getVars()[var_index:var_index + (N+1)*n]
                var_index += (N+1)*n
                z_vars = mdl_fallback.getVars()[var_index:var_index + N*N_z]
                var_index += N*N_z
                u_vars = mdl_fallback.getVars()[var_index:var_index + N*m]
                var_index += N*m

                delta_vars = mdl_fallback.getVars()[var_index:var_index + N*N_bin]
        
                x_next = np.array([x_vars[j].X for j in range(n)])  # Updated state

                u0 = np.array([u_vars[j].X for j in range(m)])  # First control action
                delta0 = np.array([delta_vars[j].X for j in range(N_bin)])  # First delta
                z0 = np.array([z_vars[j].X for j in range(N_z)])  # First z
        

                # Simulate system response
                x_next = simulate_system(x_current, u0, delta0, z0, B1, B2, B3, B5, Amld)

                # Update current state and index for the next iteration
                x_current = x_next  # Update current state for the next iteration
                index += 1  # Move to the next data point to ensure data shifts

                #print(f"Time step {i+1}, State: {x_current} (Fallback to hybrid MPC)")
                
    avg_cost = np.mean(objective_values) if objective_values else None
    print(f"Total fallbacks to hybrid MPC: {fallback_count}")
    print(f"Average Cost: {avg_cost}")
    return avg_cost, fallback_count



In [28]:
##Setup variables 
x0 = np.random.rand(1,1)*225+25 #minimum battery level is 25
N = 25
index = np.random.randint(cbuy.shape[0] - N)  # Ensure there is enough data for the horizon
models = ['transformers_here]
time_steps = 150


In [ ]:
import numpy as np
import time  # Importing the time module to measure execution time

# Initialization of parameters
N = 25
time_steps = 150
models = [shallow, shallow2, shallow_model3, backup_model4]
total_cost_difference = 0
total_avg_cost_clMPC = 0
total_avg_cost_tfMPC = 0
num_successful_simulations = 0
total_time_clMPC = 0  # Total time for classical control simulations
total_time_tfMPC = 0  # Total time for transformer-based simulations
total_fallbacks = 0  # Total fallbacks for all runs
num_simulations = 150

# Loop over the number of experiments or simulations
for _ in range(num_simulations):
    x0 = np.random.rand(1, 1) * 225 + 25  # Random initial state within the range
    index = np.random.randint(0, cbuy.shape[0] - (N+time_steps -1))  # Ensure index is within bounds
    
    net_power = power_load - power_res  
    
    # Timing the classical closed-loop MPC
    start_time_clMPC = time.time()
    avg_cost_clMPC = closed_loop_mpc(x0, N, cbuy, csell, cprod, net_power, time_steps, B1, B2, B3, B5, Amld, index)
    end_time_clMPC = time.time()
    elapsed_time_clMPC = end_time_clMPC - start_time_clMPC
    total_time_clMPC += elapsed_time_clMPC

    # Timing the transformer-based MPC
    start_time_tfMPC = time.time()
    avg_cost_tfMPC, fallbacks_tf_mpc = closed_loop_mpc_transformer1(x0, N, cbuy, csell, cprod, net_power, time_steps, B1, B2, B3, B5, 
                                                                   Amld, models, global_mean, global_std, index)
    end_time_tfMPC = time.time()
    elapsed_time_tfMPC = end_time_tfMPC - start_time_tfMPC
    total_time_tfMPC += elapsed_time_tfMPC

    total_fallbacks += fallbacks_tf_mpc
    cost_difference = ((avg_cost_tfMPC - avg_cost_clMPC) / avg_cost_tfMPC) * 100
    total_cost_difference += cost_difference
    total_avg_cost_clMPC += avg_cost_clMPC
    total_avg_cost_tfMPC += avg_cost_tfMPC
    num_successful_simulations += 1

# Calculate averages for all runs
average_avg_cost_clMPC = total_avg_cost_clMPC / num_simulations
average_avg_cost_tfMPC = total_avg_cost_tfMPC / num_simulations
average_fallbacks = total_fallbacks / num_simulations

# Print results
print("Closed Loop MPC Average Cost:", average_avg_cost_clMPC)
print("Transformer Model Average Cost:", average_avg_cost_tfMPC)
print("Average Cost Difference (TF Model vs. Classical):", total_cost_difference / num_simulations, "%")
print("Average Execution Time (Classical MPC):", total_time_clMPC / num_simulations, "seconds")
print("Average Execution Time (Transformer MPC):", total_time_tfMPC / num_simulations, "seconds")
print("Average Number of Fallbacks:", average_fallbacks)
